In [0]:
import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob


import spacy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
import json

from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm


from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

In [0]:
#@title Enter Twitter Credentials
TWITTER_KEY = 'wdxfS7Dsv3YtVK7pj5EfWv4rS' #@param {type:"string"}
TWITTER_SECRET_KEY = 'HnkTGH0KOLNUwFclKNAiBHS2pEyo4MXfvd535ZQKc7TbCTg0pf' #@param {type:"string"}

In [0]:
# Authenticate
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)


In [36]:
#@markdown ###Get All tweets

searchQuery = "PSG "
tweetsPerQry = 100  # this is the max the API permits

maxTweets = 10000
new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
topic = searchQuery[0:searchQuery.find('-')].capitalize().strip()
print(topic)

Psg


In [37]:
#@markdown ### Twitter json format
print(json.dumps(new_tweets[3]._json,indent=8))


{
        "created_at": "Fri Mar 06 22:48:07 +0000 2020",
        "id": 1236061024576552961,
        "id_str": "1236061024576552961",
        "text": "RT @brfootball: A year ago today, @MarcusRashford's 94th-minute penalty knocked PSG out of the Champions League.\n\nNeymar couldn't believe i\u2026",
        "truncated": false,
        "entities": {
                "hashtags": [],
                "symbols": [],
                "user_mentions": [
                        {
                                "screen_name": "brfootball",
                                "name": "B/R Football",
                                "id": 1561123663,
                                "id_str": "1561123663",
                                "indices": [
                                        3,
                                        14
                                ]
                        },
                        {
                                "screen_name": "MarcusRashford",
                   

In [0]:
tweet_list = []
for tweet in range(len(new_tweets)):
  retweet_text = None
  if "RT" in new_tweets[tweet]._json['text']:
    if 'retweeted_status' in new_tweets[tweet]._json.keys():
      retweet_text = new_tweets[tweet]._json['retweeted_status']['text']

  topic = searchQuery[0:searchQuery.find('-')].capitalize().strip()
  tweet_user = new_tweets[tweet]._json['user']['name']
  isRetweet = new_tweets[tweet]._json["retweeted"]
  tweet_list.append([tweet_user, topic, isRetweet, retweet_text])

df = pd.DataFrame(tweet_list,columns=["user", "topic", "isRetweet", "retweeted"])
      

In [39]:
df


,user,topic,isRetweet,retweeted
0,Toni Grinton,Psg,False,Breaking | Strasbourg vs PSG postponed owing t...
1,Glenn,Psg,False,"A year ago today, @MarcusRashford's 94th-minut..."
2,cet homme,Psg,False,None
3,Dybala_soumillion,Psg,False,"A year ago today, @MarcusRashford's 94th-minut..."
4,Samuel J. (THE🇦🇺OLFAN),Psg,False,None
...,...,...,...,...
95,Nigel Refalo,Psg,False,None
96,BigDej,Psg,False,"A year ago today, @MarcusRashford's 94th-minut..."
97,Anfield Watch,Psg,False,None
98,KaRiM ❤👑,Psg,False,None


In [0]:
list_df = df.get("retweeted").tolist()
df_filter = [ el for el in list_df if el != None]

In [41]:
df_filter

['Breaking | Strasbourg vs PSG postponed owing to coronavirus concerns. More follows.',
 "A year ago today, @MarcusRashford's 94th-minute penalty knocked PSG out of the Champions League.\n\nNeymar couldn't b… https://t.co/uA091qQv3q",
 "A year ago today, @MarcusRashford's 94th-minute penalty knocked PSG out of the Champions League.\n\nNeymar couldn't b… https://t.co/uA091qQv3q",
 "Regarding the #GreeceAttacksRefugees .\nIn order for somebody to attack, he must cross the border line.\nGreece hasn'… https://t.co/zcBuMc8g0E",
 "A year ago today, @MarcusRashford's 94th-minute penalty knocked PSG out of the Champions League.\n\nNeymar couldn't b… https://t.co/uA091qQv3q",
 "A year ago today, @MarcusRashford's 94th-minute penalty knocked PSG out of the Champions League.\n\nNeymar couldn't b… https://t.co/uA091qQv3q",
 'Exactly 2 years ago today, @Cristiano Ronaldo destroyed PSG at Parc Des Princes.🔥🔥🔥 https://t.co/HUlLqeiCYP',
 'Making memories with ALL. A lucky member flew by private jet to

In [42]:
!pip install transformers

In [0]:
import torch
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

In [0]:
modelname = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(modelname)

In [0]:
text_tokenize = []
count = 0
for sentence in df_filter:
  text_tokenize.append(tokenizer.tokenize(sentence))
  count += 1

In [46]:
print(text_tokenize)

[['Breaking', 'Ġ|', 'ĠStras', 'bourg', 'Ġvs', 'ĠPS', 'G', 'Ġpostponed', 'Ġowing', 'Ġto', 'Ġcoron', 'av', 'irus', 'Ġconcerns', '.', 'ĠMore', 'Ġfollows', '.'], ['A', 'Ġyear', 'Ġago', 'Ġtoday', ',', 'Ġ@', 'Marcus', 'R', 'ash', 'ford', "'s", 'Ġ94', 'th', '-', 'minute', 'Ġpenalty', 'Ġknocked', 'ĠPS', 'G', 'Ġout', 'Ġof', 'Ġthe', 'ĠChampions', 'ĠLeague', '.', 'Ċ', 'Ċ', 'N', 'ey', 'mar', 'Ġcouldn', "'t", 'Ġb', 'âĢ¦', 'Ġhttps', '://', 't', '.', 'co', '/', 'u', 'A', '09', '1', 'q', 'Q', 'v', '3', 'q'], ['A', 'Ġyear', 'Ġago', 'Ġtoday', ',', 'Ġ@', 'Marcus', 'R', 'ash', 'ford', "'s", 'Ġ94', 'th', '-', 'minute', 'Ġpenalty', 'Ġknocked', 'ĠPS', 'G', 'Ġout', 'Ġof', 'Ġthe', 'ĠChampions', 'ĠLeague', '.', 'Ċ', 'Ċ', 'N', 'ey', 'mar', 'Ġcouldn', "'t", 'Ġb', 'âĢ¦', 'Ġhttps', '://', 't', '.', 'co', '/', 'u', 'A', '09', '1', 'q', 'Q', 'v', '3', 'q'], ['Regarding', 'Ġthe', 'Ġ#', 'G', 'ree', 'ce', 'Att', 'acks', 'Ref', 'uge', 'es', 'Ġ.', 'Ċ', 'In', 'Ġorder', 'Ġfor', 'Ġsomebody', 'Ġto', 'Ġattack', ',', 'Ġhe', 'Ġm